In [1]:
# Импортируем все необходимые библиотеки
import pandahouse
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

In [2]:
# Подключаемся к clickhouse
connection = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'simulator_20220620',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
# Вытаскиваем пользователей
q = """
SELECT distinct user_id
FROM {db}.feed_actions
WHERE toDate(time) >= '2022-05-24' and toDate(time) <= '2022-05-30'
"""

users_df = pandahouse.read_clickhouse(q, connection=connection)
users_df.head()

,user_id
0,115269
1,122097
2,132354
3,134989
4,135083


In [4]:
# Делаем группы
q = """
SELECT xxHash64(toString(user_id)||'my_salt')%5 as exp_group, 
    uniqExact(user_id)
FROM {db}.feed_actions 
WHERE toDate(time) >= '2022-05-24' and toDate(time) <= '2022-05-30' 
group by exp_group 
"""

pandahouse.read_clickhouse(q, connection=connection)

,exp_group,uniqExact(user_id)
0,0,10091
1,1,9976
2,2,9966
3,3,10140
4,4,9976


In [5]:
# Смотрим на нужные нам группы (1 и 2), записываем их в датафрейм

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) >= '2022-05-24' and toDate(time) <= '2022-05-30' 
    and exp_group in (1,2)
GROUP BY exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection=connection)

df.groupby('exp_group').count()

,user_id,likes,views,ctr
exp_group,,,,
1,10079,10079,10079,10079
2,9952,9952,9952,9952


In [6]:
# Считаем общий CTR в контрольной группе
control_ctr = df[df.exp_group == 1].likes.sum()/df[df.exp_group == 1].views.sum()
control_ctr

0.20802680490126244

In [7]:
# Cчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠
df['linearized_likes']=df.likes - control_ctr*df.views
df

,exp_group,user_id,likes,views,ctr,linearized_likes
0,1,109963,3,15,0.200000,-0.120402
1,1,26117,32,141,0.226950,2.668221
2,1,138232,18,73,0.246575,2.814043
3,1,26295,33,122,0.270492,7.620730
4,1,18392,7,32,0.218750,0.343142
...,...,...,...,...,...,...
20026,1,132567,45,296,0.152027,-16.575934
20027,1,30872,11,59,0.186441,-1.273581
20028,2,25383,42,127,0.330709,15.580596
20029,2,139020,24,114,0.210526,0.284944


In [8]:
# Cравниваем t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠
stats.ttest_ind(df[df.exp_group == 1].linearized_likes,
                df[df.exp_group == 2].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)

In [9]:
# Делаем обычный t-тест Стьюдента
stats.ttest_ind(df[df.exp_group == 1].ctr,
                df[df.exp_group == 2].ctr,
                equal_var=False)

Ttest_indResult(statistic=0.7094392041270486, pvalue=0.4780623130874935)

## Проанализировали тест между группами 1 и 2 по метрике линеаризованных лайков. Т-тест показал отличие между группами. P-value меньше, чем в обычном t-тесте по CTR, который отличия не показал.

In [11]:
# Делаем все то же самое, но на группах 0 и 3

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) >= '2022-05-24' and toDate(time) <= '2022-05-30' 
    and exp_group in (0,3)
GROUP BY exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection=connection)

df.groupby('exp_group').count()

,user_id,likes,views,ctr
exp_group,,,,
0,9999,9999,9999,9999
3,10049,10049,10049,10049


In [12]:
# Считаем общий CTR в контрольной группе
control_ctr = df[df.exp_group == 0].likes.sum()/df[df.exp_group == 0].views.sum()
control_ctr

0.20823612262916305

In [13]:
# Cчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠
df['linearized_likes']=df.likes - control_ctr*df.views
df

,exp_group,user_id,likes,views,ctr,linearized_likes
0,3,115383,9,30,0.300000,2.752916
1,3,123580,13,48,0.270833,3.004666
2,0,4944,8,41,0.195122,-0.537681
3,0,4504,5,15,0.333333,1.876458
4,0,121508,18,88,0.204545,-0.324779
...,...,...,...,...,...,...
20043,3,13781,10,27,0.370370,4.377625
20044,3,130651,18,98,0.183673,-2.407140
20045,0,3681,7,55,0.127273,-4.452987
20046,3,122008,5,41,0.121951,-3.537681


In [15]:
# Делаем обычный t-тест Стьюдента
stats.ttest_ind(df[df.exp_group == 0].ctr,
                df[df.exp_group == 3].ctr,
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [17]:
# Cравниваем t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠
stats.ttest_ind(df[df.exp_group == 0].linearized_likes,
                df[df.exp_group == 3].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=-15.214995460903827, pvalue=5.4914249479690016e-52)

In [20]:
5.4914249479690016e-52<1.055849414662529e-43

True

## В группах 0 и 3 оба t-теста показали отличие между группами. P-value по метрике линеаризованных лайков меньше, чем в обычном t-тесте по CTR.